<a href="https://colab.research.google.com/github/Ranjit07K/Atm1/blob/main/Emai_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client imapclient


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 6.5 MB/s eta 0:00:00


In [2]:
pip install imapclient elasticsearch fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00


In [7]:
from google_auth_oauthlib.flow import InstalledAppFlow
import json

def get_oauth_token():
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json",
        scopes=["https://mail.google.com/"]
    )

    # Use local server authentication, but don't open browser automatically
    authorization_url, _ = flow.authorization_url(prompt='consent')

    # Print the authorization URL to the console
    print(f"Go to this URL in your browser: {authorization_url}")
    print("Enter the authorization code: ", end="")
    authorization_code = input()  # Ask the user to input the code

    # Fetch the token using the authorization code
    flow.fetch_token(code=authorization_code)

    # Save token for future use
    with open("token.json", "w") as token_file:
        json.dump({
            "token": flow.credentials.token,
            "refresh_token": flow.credentials.refresh_token,
            "client_id": flow.credentials.client_id,
            "client_secret": flow.credentials.client_secret
        }, token_file)

    print("✅ OAuth token saved to token.json")

if __name__ == "__main__":
    get_oauth_token()

Go to this URL in your browser: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=495004704534-gug16f7aicubh62n1bbke0noshlhpn7a.apps.googleusercontent.com&scope=https%3A%2F%2Fmail.google.com%2F&state=szwixBquUrbarpFmNbZY2zNBnqXSTa&prompt=consent&access_type=offline
Enter the authorization code: 

KeyboardInterrupt: Interrupted by user

In [4]:
pip install google-auth google-auth-oauthlib google-auth-httplib2


In [ ]:
import imaplib
import json
import base64
import requests

IMAP_SERVER = "imap.gmail.com"

def refresh_access_token():
    with open("token.json", "r") as token_file:
        token_data = json.load(token_file)

    response = requests.post(
        "https://oauth2.googleapis.com/token",
        data={
            "client_id": token_data["client_id"],
            "client_secret": token_data["client_secret"],
            "refresh_token": token_data["refresh_token"],
            "grant_type": "refresh_token"
        }
    )

    new_token = response.json()["access_token"]
    token_data["token"] = new_token

    # Save the new token
    with open("token.json", "w") as token_file:
        json.dump(token_data, token_file)

    return new_token

def login_imap():
    access_token = refresh_access_token()
    auth_string = f"user={EMAIL}\x01auth=Bearer {access_token}\x01\x01"
    auth_encoded = base64.b64encode(auth_string.encode("utf-8")).decode("utf-8")

    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.authenticate("XOAUTH2", lambda x: auth_encoded)
    mail.select("INBOX")

    return mail

if __name__ == "__main__":
    EMAIL = "k.ranjithraw@gmail.com"  # Replace with your Gmail address
    mail = login_imap()
    print("IMAP OAuth2 login successful!")
